In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [16]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 14:00:00+00:00


In [17]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [18]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-04 08:22:50,153 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:22:50,209 INFO: Initializing external client
2025-03-04 08:22:50,211 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:22:51,473 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


In [19]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


In [20]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,28,0.0,2025-03-04 01:00:00+00:00
1,128,0.0,2025-03-04 01:00:00+00:00
2,7,2.0,2025-03-04 01:00:00+00:00
3,130,1.0,2025-03-04 01:00:00+00:00
4,19,0.0,2025-03-04 01:00:00+00:00
...,...,...,...
1752,138,181.0,2025-03-04 11:00:00+00:00
1753,186,191.0,2025-03-04 11:00:00+00:00
1754,85,0.0,2025-03-04 11:00:00+00:00
1755,37,2.0,2025-03-04 11:00:00+00:00


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  1757 non-null   int32                  
 1   predicted_demand    1757 non-null   float64                
 2   pickup_hour         1757 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 34.4 KB


In [24]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 08:23:54,431 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:23:54,440 INFO: Initializing external client
2025-03-04 08:23:54,441 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:23:55,803 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Current UTC time: 2025-03-04 13:23:54.431982+00:00
Next hour: 2025-03-04 14:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 14:00
Found 0 records


In [25]:
now = datetime.now(timezone.utc)

In [26]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  1757 non-null   int32                  
 1   predicted_demand    1757 non-null   float64                
 2   pickup_hour         1757 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 34.4 KB


In [28]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [29]:
dt = current_date.ceil('h')


In [30]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
# query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-04 08:24:31,039 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:24:31,052 INFO: Initializing external client
2025-03-04 08:24:31,053 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:24:32,187 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.53s) 


In [31]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.19s) 


,pickup_location_id,predicted_demand,pickup_hour


In [32]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,28,0.0,2025-03-04 01:00:00+00:00
1,128,0.0,2025-03-04 01:00:00+00:00
2,7,2.0,2025-03-04 01:00:00+00:00
3,130,1.0,2025-03-04 01:00:00+00:00
4,19,0.0,2025-03-04 01:00:00+00:00
...,...,...,...
2003,142,140.0,2025-03-04 14:00:00+00:00
2004,41,6.0,2025-03-04 14:00:00+00:00
2005,226,2.0,2025-03-04 14:00:00+00:00
2006,190,0.0,2025-03-04 14:00:00+00:00


In [33]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [34]:
predictions = fetch_next_hour_predictions()

2025-03-04 08:25:01,768 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-03-04 08:25:01,781 INFO: Initializing external client
2025-03-04 08:25:01,783 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:25:03,400 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Current UTC time: 2025-03-04 13:25:01.767057+00:00
Next hour: 2025-03-04 14:00:00+00:00
Found 0 records


In [35]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-04 08:25:07,816 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 08:25:07,828 INFO: Initializing external client
2025-03-04 08:25:07,829 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 08:25:09,265 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


In [36]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 


In [37]:
df["pickup_hour"].max()

Timestamp('2025-03-04 14:00:00+0000', tz='Etc/UTC')

In [38]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 14:00:00+00:00


In [39]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
1757,143,39.0,2025-03-04 14:00:00+00:00
1758,242,0.0,2025-03-04 14:00:00+00:00
1759,27,0.0,2025-03-04 14:00:00+00:00
1760,8,0.0,2025-03-04 14:00:00+00:00
1761,115,0.0,2025-03-04 14:00:00+00:00
...,...,...,...
2003,142,140.0,2025-03-04 14:00:00+00:00
2004,41,6.0,2025-03-04 14:00:00+00:00
2005,226,2.0,2025-03-04 14:00:00+00:00
2006,190,0.0,2025-03-04 14:00:00+00:00
